## DIA 130: Agregar Búsqueda por Palabra Clave en el Historial de Reportes

Implementar una barra de búsqueda por palabra clave que permita al usuario encontrar reportes según:

🔤 Título del reporte

📝 Comentarios u observaciones

🏷️ Categoría (opcionalmente)

🧩 1. Modificar la ruta historial_reportes para aceptar query
python
Copiar
Editar
@app.route('/historial')
@login_required
def historial_reportes():
    query = request.args.get('q', '').strip()
    categoria_filtro = request.args.get('categoria')

    # Obtener categorías únicas
    categorias = db.session.query(Reporte.categoria).filter_by(user_id=current_user.id).distinct().all()
    categorias = [c[0] for c in categorias if c[0]]

    # Base de reportes
    reportes = Reporte.query.filter_by(user_id=current_user.id)

    if query:
        # Buscar en título o comentario (ilike para que no sea sensible a mayúsculas)
        reportes = reportes.filter(
            db.or_(
                Reporte.comentario.ilike(f'%{query}%'),
                Reporte.categoria.ilike(f'%{query}%'),
                Reporte.url.ilike(f'%{query}%'),
                Reporte.fecha_generado.ilike(f'%{query}%')
            )
        )

    if categoria_filtro:
        reportes = reportes.filter(Reporte.categoria.ilike(f'%{categoria_filtro}%'))

    reportes = reportes.order_by(Reporte.fecha_generado.desc()).all()
    return render_template("historial_reportes.html", reportes=reportes, categorias=categorias)
🧩 2. Agregar el formulario de búsqueda en historial_reportes.html
Coloca arriba de la tabla de reportes:

html
Copiar
Editar
<form method="GET" action="{{ url_for('historial_reportes') }}">
  <input type="text" name="q" placeholder="Buscar por título, comentario, categoría..." value="{{ request.args.get('q', '') }}">
  <button type="submit">🔍 Buscar</button>
  <a href="{{ url_for('historial_reportes') }}">Limpiar</a>
</form>